In [18]:
import argparse


In [19]:
class Node:
    def __init__(self, name, father, isleaf=False, isand=None):
        self.name = name
        self.father=None
        self.lchild = None
        self.rchild = None
        self.isleaf=isleaf #是否是叶子节点
        self.isand=isand #如果是分支节点 与或判断
        self.solvable = None #可解与否

In [20]:
class Tree:
    def __init__(self):
        self.root = None
        
    #构造与或树
    def add(self, name,father,isleaf,isand):
        node = Node(name, father, isleaf, isand)
        if self.root is None:
            self.root = node
        else:
            q = [self.root]

            while True:
                pop_node = q.pop(0)
                if pop_node.name==father:
                    
                    if pop_node.lchild is None:
                        pop_node.lchild = node
                        node.father = pop_node
                        return
                    elif pop_node.rchild is None:
                        pop_node.rchild = node
                        node.father = pop_node
                        return
                else:
                    if pop_node.lchild!=None:
                        q.append(pop_node.lchild)
                    if pop_node.rchild!=None:
                        q.append(pop_node.rchild)
    #宽度优先遍历
    def BFS( self,root ):
        if ((self.root is None) or (self.root.isleaf == True ) ) :
            return None
        #open and close 表
        open1 = [self.root]
        close = []
        flag=None #找到解树标志 false 无解 true 有解 none 未知
        #开始遍历
        while open1 != []:
            
            pop_node = open1.pop(0)
            close.append(pop_node)
            # 遍历到叶子节点 叶子节点 可解 True 然后标志整棵树
            if pop_node.isleaf == True:
                pop_node.solvable = True
                flag = self.mark(root)   #是否找到解树
                self.remove1(root,open1) # 删除open表 先辈可解节点 和不可解节点
            else:                
                if pop_node.lchild is not None:
                    open1.append(pop_node.lchild)
                if pop_node.rchild is not None:
                    open1.append(pop_node.rchild)
                if ((pop_node.lchild ==None) and (pop_node.rchild ==None)) :# 无终叶节点的分支节点 置false
                    pop_node.solvable = False
                    flag = self.mark(root)   #更新与或树标记 返回 是否找到解树
                    self.remove1(root,open1) # 删除open表 先辈可解节点 和不可解节点
        #if flag==True:
         #   return close
        if flag==True:
            return close
        else :
            return None
    #深度优先遍历
    def DFS(self,root):
        if ((self.root is None) or (self.root.isleaf == True ) ) :
            return None
        #open and close 表
        open1 = [self.root]
        close = []
        flag=None #找到解树标志 false 无解 true 有解 none 未知
        #开始遍历
        while open1 != []:
            
            pop_node = open1.pop()
            close.append(pop_node)
            #深度过大标记无解，不遍历该子树
            if (self.deep(pop_node)>10):
                pop_node.solvable = False
                flag = self.mark(root)   #是否找到解树
                self.remove1(root,open1) # 删除open表 先辈可解节点 和不可解节点
                
            else: #深度在要求之内
                
                # 遍历到叶子节点 叶子节点 可解 True 然后标志整棵树
                if pop_node.isleaf == True:
                    pop_node.solvable = True
                    flag = self.mark(root)   #是否找到解树
                    self.remove1(root,open1) # 删除open表 先辈可解节点 和不可解节点
                else:                
                    #与宽度优先不同，深度优先先将右子树 加入list ，因为pop()是取list最后元素，左子树先取应在list最后的元素
                    if pop_node.rchild is not None:
                        open1.append(pop_node.rchild)
                    if pop_node.lchild is not None:
                        open1.append(pop_node.lchild)
                    if ((pop_node.lchild ==None) and (pop_node.rchild ==None)) :# 无终叶节点的分支节点 置false
                        pop_node.solvable = False
                        flag = self.mark(root)   #更新与或树标记 返回 是否找到解树
                        self.remove1(root,open1) # 删除open表 先辈可解节点 和不可解节点
        #有解 返回解
        if flag==True:
            return close
        else :
            return None
        
    
    #后序遍历进行标记可解节点与不可解节点 同时进行
    def mark(self,root):
        if root == None:
            return None 
        if (root.lchild == None) and (root.lchild == None):
            return root.solvable

        left = self.mark(root.lchild)
        right = self.mark(root.rchild)
        #只对未标记节点进行标记
        if root.solvable == None :
            if root.isand == True :
                if ((left == True) and (right == True) ):
                    root.solvable = True        
                if ((left == False) or (right == False) ):
                    root.solvable = False
            if root.isand == False :
                if ((left == True) or (right == True) ):
                    root.solvable = True 
                if ((left == False) and (right == False) ):
                    root.solvable = False

        return root.solvable
    #判断先辈节点是否为 可解 or 不可解
    def fathersolvable(self,node):
        if node.father==None:
            return None
        else :
            if node.father.solvable == None:
                
                return self.fathersolvable(node.father)
            else:
                return node.father.solvable
    #删除open表中先辈节点是可解或者不可解的节点
    def remove1(self,root,open1):
        for node in open1:
            if self.fathersolvable(node)!=None:
                open1.remove(node)
    def deep(self,node):
        deep=1
        while node.father!=None:
            node=node.father
            deep=deep+1
        return deep
        
        

In [24]:
def main():
    t1=build()
    t2=build()
    close1=t1.BFS(t1.root)
    close2=t2.DFS(t2.root)
    #for test
    print(t1.root.rchild.lchild.rchild.solvable)
    print('BFS result')
    printclose(close1)
    print('DFS result')
    printclose(close2)
def build():
    
    t=Tree()
    #add(self, item,father,isleaf，isand)
    #构造实例树
    t.add('1',None,False,True)
    t.add('2','1',False,False)
    t.add('3','2',False,False)
    t.add('4','3',False,False)
    t.add('C','4',False,False)
    t.add('A','2',False,False)
    t.add('5','3',False,True)
    t.add('6','1',False,False)
    t.add('7','6',False,True)
    t.add('8','7',False,True)
    t.add('B','6',False,False)
    t.add('t1','5',True,None)
    t.add('t2','5',True,None)
    t.add('t3','8',True,None)
    t.add('t4','8',True,None)
    t.add('t5','7',True,None)
    return t
def printclose(close):
    #输出close中所有可解节点
    if close==None:
        print('No Solusion')
    else:
        for node in close:
            if node.solvable==True:
                print(node.name)
main()


True
BFS result
1
2
6
3
7
5
8
t5
t1
t2
t3
t4
DFS result
1
2
3
5
t1
t2
6
7
8
t3
t4
t5
